# Logistička regresija

In [ ]:
from testbench import Testbench

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
Testbench.author('Petar Petrović')

## Učitavanje podataka

Dataset sadrži dve komponente koje nam daje PCA (nazvane su PC1 i PC2) i kategoriju težine (nazvana je NObeyesdad, preimenovaćemo je u Obese) koju koristimo kao labelu. Težina ima više vrednosti koje su pretvorene u numeričke (0-3) i predstavljaju redom neuhranjenost, normalnu težinu, prekomernu težinu i gojaznost. Pošto je logistička regresija binarni klasifikator, radićemo predikciju samo da li je osoba gojazna ili ne, odnosno prve 3 kategorije ćemo spojiti u jednu (nije gojazna). 

In [ ]:
def load_data(name: str):
    df = pd.read_csv(name)
    clean_df = pd.DataFrame()
    
    # PC1 i PC2 kolone samo kopiramo
    for col in ["PC1", "PC2"]:
        clean_df[col] = df[col]
        
    # sa proverom jednakosti dobijamo True/False, pretvaranjem u int dobijamo 1/0 sto nam treba
    targets = (df["NObeyesdad"] == 3).astype(int).rename("Obese")

    return clean_df, targets

In [ ]:
features, labels = load_data("../datasets/obesity_pca.csv")

## Vizualizacija podataka

Za početak možemo pogledati kako izgledaju naši podaci u 2D prostoru. Na osama su vrednosti izdvojenih komponenti, svaka tačka je jedna osoba iz dataseta a njena boja pokazuje da li je gojazna (narandžasta ako jeste ili plava ako nije).

In [ ]:
plt.figure(figsize = (12, 8))
data = pd.concat([features, labels], axis=1)
sns.scatterplot(data=data, x="PC1", y="PC2", hue="Obese")
plt.show()

## Hipoteza

Hipoteza je glavni deo našeg modela, to je funkcija koja na osnovu težina (koje će naš algoritam da "nauči") i featura osobe vraća "mišljenje" našeg modela o gojaznosti te osobe. Hipoteza vraća realan broj između 0 i 1, a tek nakon treniranja ćemo ga pretvoriti u klasu (0 ako je ispod 0.5, 1 ako je iznad). To nam omogućava da model postepeno uči, jer ne mora da pravi skokove između 0 i 1 već može polako da prelazi.

Nakon što se vrednosti featura pomnože sa odgovarajućim težinama i kad se te vrednosti saberu, primenjujemo sigmoidnu funkciju. Njen izlaz je uvek između 0 i 1 (najveće negativne vredosti idu u 0 a pozitivne u 1) što nam i treba.

In [ ]:
# X - lista featura osoba
# theta - težine
# funkcija vraća listu hipoteza, za svaku osobu po jednu predikciju
def hypothesis(X, theta):
    # mnozenje vrednosti featura sa tezinama i njihovo sabiranje, predstavljeno u obliku matricnog proizvoda
    z = np.dot(theta, X.T)
    # ne zelimo da dobijemo tacno 0 ili 1 jer ce nam kasnije to praviti problem u logaritmu,
    # zato se ogranicimo na opseg od 0.000001 do 0.9999999
    return np.clip(1/(1+np.exp(-(z))), 0.000001, 0.9999999)

## Cost funkcija

Cost funkcija (nekad se zove i loss funckija) je funkcija čiji minimum ćemo tražiti, i treba da predstavlja grešku hipoteze tj. razliku predviđenih i stvarnih klasa na celom datasetu. Minimizacijom ove funkcije minimizovaće se i ta razlika tj. greška modela.

Najjednostavnije bi bilo uzeti prosečnu apsolutnu vrednost razlike predviđanja i stvarnih labela, međutim postoje i druge funkcije sa kojima se postiže brža konvergencija algoritma (brži dolazak do minimuma). Neke od često korišćenih su mean square error (MSE) - umesto apsolutne vrednosti razlike koristi se njen kvadrat; i cross-entropy, koja u suštini predstavlja prosečan logaritam greške.

In [ ]:
# y - stvarne klase osoba
# y1 - hipoteze
# funkcija vraća prosecan cost na celom datasetu
def cost(y, h):
    # racunamo cross entropy
    return -(1/len(y)) * np.sum(y*np.log(h) + (1-y)*np.log(1-h))

## Gradient descent

Glavni deo logističke regresije koji zapravo "uči" težine. Radi tako što izračuna gradijent costa po theta i "pomeri" theta u pravcu tog gradijenta. Koliko će se pomeriti zavisi od learning rate-a, a koliko puta će ponoviti taj postupak zavisi od zadatog broja iteracija. Veći learning rate ubrzava prilaženje minimuma ali takođe i povećava šansu da će se minimum preskočiti. Veći broj iteracija produžava vreme izvršavanja algoritma ali dozvoljava da se više približimo minimumu (ako algoritam nije iskonvergirao u ranijim iteracijama).

### Zadatak

Treba popuniti označenu liniju tako da se kompletira gradient descent.

**Pomoć:** Parcijalni izvod costa po `theta_i` je `1/m * sum((h[j] - y[j]) * X[j][i])` (`m` je broj redova u `X`; suma je po `j` od `0` do `m`).

In [ ]:
# X - lista featura osoba
# y - stvarne klase osoba
# theta - pocetne tezine
# alpha - learning rate koeficijent
# epochs - broj iteracija
# funkcija vraca vrednosti costa kroz iteracije i naucene theta koeficijente
def gradient_descent(X, y, theta, alpha, epochs):
    # racunanje pocetnog costa
    h = hypothesis(X, theta)
    cost_history = [cost(y, h)]
    
    m = len(X)
    for _ in range(0, epochs):
        h = hypothesis(X, theta)
        for i in range(0, len(theta)):
            # updateovanje theta[i]
            ##############
            # TODO Popuni
            ##############
            
        # cuvanje costa u trenutnoj iteraciji
        cost_history.append(cost(y, h))
    return cost_history, theta

### Testiranje rešenja

In [ ]:
def logistical_regression(X, y, theta, alpha, epochs):
    return gradient_descent(X, y, theta, alpha, epochs)
Testbench(logistical_regression);

## Predviđanje

Da bismo dobili predviđanje modela, treba samo da izračunamo hipotezu sa istreniranim koeficijentima i da zaokružimo vrednosti na 0 ili 1.

In [ ]:
# X - lista featura osoba
# theta - težine
# funkcija vraća listu predikcija, za svaku osobu po jednu predikciju
def predict(X, theta):
    h = hypothesis(X, theta)
    for i in range(len(h)):
        h[i]=1 if h[i]>=0.5 else 0
    return h

## Pokretanje algoritma

Sve što je preostalo je da istreniramo model, dobijemo predikcije i izračunamo tačnost modela. Za početne vrednosti koeficijenata smo potpuno nasumično uzeli 0.5, a za learning rate i broj iteracija su uzete vrednosti koje za ovaj dataset daju prihavtljivu tačnost i vreme izvršavanja. Te parametre možete menjati i videti njihov uticaj, a u sledećem odeljku je i objašnjenje kako možete doći do još boljeg izbora parametara.

In [ ]:
# pocetne vrednosti koeficijenata
theta = [0.5]*len(features.columns)
# ucenje koeficijenata
cost_history, theta = gradient_descent(features, labels, theta, 0.0001, 500)
# predikcija
y_pred = predict(features, theta)

# racunanje tacnosti
y = list(labels)
acc = np.sum([y[i] == y_pred[i] for i in range(len(y))])/len(y)
print("Tačnost:", acc * 100, "%")

## Vizualizacija greške

Jedan način provere da li algoritam dobro radi je da vidimo kako se greška (tj. vrednost cost funkcije) menja tokom iteracija. Greška bi uvek trebalo da opada kroz iteracije ako je gradient descent dobro implementiran.

Ovaj grafik nam takođe može pomoći da odredimo vrednosti za learning rate i broj iteracija. Ako greška previše sporo opada, to je znak da se learning rate možda može povećati, a ako brzo opada i konvergira, treba probati da se smanji (što može dovesti do nalaženja boljeg minimuma). Što se tiče broja iteracija, ako vidimo da greška i u poslednjim iteracijama nastavlja da opada onda ima smisla povećati broj iteracija, a ako vrlo brzo prestane da se smanjuje mogao bi se smanjiti broj i time se smanjiti dužina treniranja bez velikog uticaja na grešku.

Ovaj "recept" je samo okviran, na primer premali learning rate i premali broj iteracija daju sličan grafik (ne znate da li greška sporo opada zato što je learning rate premali, ili zato što je problem "težak" i jednostavno je potrebno više iteracija). Jedno rešenje je da se fiksira broj iteracija i prvo odredi optimalan learning rate pa tek onda da se menja broj iteracija.

In [ ]:
%matplotlib inline
plt.figure(figsize = (12, 8))
plt.plot(range(len(cost_history)), cost_history)
plt.ylabel("Greška")
plt.xlabel("Iteracija")
plt.show()